In [2]:
import numpy as np
import matplotlib.pyplot as plt
import stim
from lib.stabilizer import measurement_gadgets, StabilizerCode, stabilizer_circuits
from lib.color_compass import Lattice2D, compass_to_surface
from lib.decoder import checkmatrix,pL_from_checkmatrix
from lib.stim2pymatching import estimate_pL_noisy_graph
import stimcirq
from typing import *
from cirq.contrib.svg import SVGCircuit

The basic learning loop is as follows 

0) Fix a hidden Pauli noise model for the gates and measurements
1) Pick a compass code and get its stabilizers and logical operators
2) Construct a noisy stim circuit for the code under the noise model 
3) Simulate and get logical error rate
4) Go to 1) and repeat

In [4]:
# Construction of a Pauli noise model

class PauliNoiseModel():
    """
    Constructs noisy Stim circuits from 2D Compass Code stabilizers and logical observables

    A noise model is defined as mapping a perfect operation to an imperfect operation
    """
    def __init__(self, one_qb_gate_rates : List[float], two_qb_gate_rates : List[float], meas_error_rate : float):
        self.one_qb_gate_rates = one_qb_gate_rates
        assert(len(one_qb_gate_rates) == 3)
        self.two_qb_gate_rates = two_qb_gate_rates
        assert(len(two_qb_gate_rates) == 15)
        self.meas_error_rate = meas_error_rate

    def one_qb_pauli_noise(self) -> str:
        """ 
        Returns a string representing a single qubit Stim Pauli error channel
        """
        channel_str = 'PAULI_CHANNEL_1({},{},{})'.format(*self.one_qb_gate_rates)
        return channel_str 
    
    def two_qb_pauli_noise(self) -> str:
        """ 
        Returns a string representing a two qubit Stim Pauli error channel
        """
        channel_str = 'PAULI_CHANNEL_2({},{},{},{},{},{},{},{},{},{},{},{},{},{},{})'.format(*self.two_qb_gate_rates)
        return channel_str 
    
    def measurement_gadget(self, pauli_observable : str):
        """ 
        Stim gadget to directly measure the specified 'pauli_observable'
        """
        meas_circ = ''
        x_meas_pos = ''
        y_meas_pos = '' 
        z_meas_pos = ''
        pos = {'I' : [], 'X' : [], 'Y' : [], 'Z' : []}
        for i, pauli in enumerate(pauli_observable):
            if pauli == 'X':
                x_meas_pos += f' {i}'
            elif pauli == 'Y':
                y_meas_pos += f' {i}'
            elif pauli == 'Z':
                z_meas_pos += f' {i}'
            else:
                pass 

        meas_circ += f'MX({self.meas_error_rate})' + (x_meas_pos * len(x_meas_pos)) + '\n' + f'MY({self.meas_error_rate})' + (y_meas_pos * len(y_meas_pos)) + '\n' + f'MZ({self.meas_error_rate})' + (z_meas_pos * len(z_meas_pos)) + '\n'
        return stim.Circuit(meas_circ)
    
    def stabilizer_gadget(self, stabilizer_in, ancilla_index, construction='cnot'):
        """
        Input:
            stabilizer: a single stabilizer written in terms of {I/_,X,Y,Z}
            construction: direct or hadamard:
                1) `cnot` using only CNOTs from data to ancilla along with single qubit gates
                    - H then S    : rotates Z basis -> Y basis
                    - S_dag then H: rotates Y basis -> Z basis
                    verifiable via checking that: Y stabilizer == kron(S@H,I) @ CNOT @ kron(H@S_dag)
                2) `hadamard` using H gates on ancilla and C-Pauli from ancilla to data
        Output:
            Measurement gadget

        the data qubits that are indicated in the stabilizer appear first
        the ancilla index starts from 0, which is the first ancilla qubit after the data 
        """
         # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')

        

    def stabilizer_gadget_v2(self, stabilizer_in):
        """
        Use Stim's built in 'MPP' function
        (IS THIS PREFERRED OVER SPLITTING UP MEASUREMENTS INTO CONSTITUENT PARTS AND APPLYING CIRCUIT-LEVEL NOISE?)
        """
        # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = f'MPP({self.meas_error_rate}) '
        for i, pauli in enumerate(stabilizer):
            if (pauli != 'I'):
                circ_string += f'{pauli}{i}*'
        circ_string = circ_string[:-1] + '\n'
        return stim.Circuit(circ_string)
        

        

        

IndentationError: unexpected indent (1984052441.py, line 50)

In [ ]:
"""noise model construction""" 

class NoisyGateSet():
    """
    Constructs noisy stim circuits from stabilizers and logical operations
    
    A noise model takes an operation and outputs a noisy gadget 
    """
    def __init__(self):
        self.gate_1qb_rates = [0, 0, 0]
        self.gate_2qb_rates = [0]*15
        self.meas_error_rate = 0
        
    def set_1qb_rates(self, rates):
        self.gate_1qb_rates = rates.copy()
    
    def set_2qb_rates(self, rates):
        self.gate_2qb_rates = rates.copy()
        
    def set_meas_error(self, rate):
        self.meas_error_rate = rate
        
    def noise_string_1qb(self):
        """returns stim string for 1qb Pauli channel"""
        noise_string = 'PAULI_CHANNEL_1('
        for r in self.gate_1qb_rates:
            noise_string += f'{r},'
        noise_string = noise_string[:len(noise_string)-1] # remove last comma
        noise_string += ') '
        return noise_string
    
    def noise_string_2qb(self):
        """returns stim string for 2qb Pauli channel"""
        noise_string = 'PAULI_CHANNEL_2('
        for r in self.gate_2qb_rates:
            noise_string += f'{r},'
        noise_string = noise_string[:len(noise_string)-1] # remove last comma
        noise_string += ') '
        return noise_string
    
    def measurement_gadget(self, pauli_observable):
        """Gadget to directly measure the observable with measurement noise"""
        circ_string = ''
        xpart = ''
        ypart = ''
        zpart = ''
        for i, pauli in enumerate(pauli_observable):
            if pauli == 'X':
                xpart += f'{i} '
            elif pauli == 'Y':
                ypart += f'{i} '
            elif pauli == 'Z':
                zpart += f'{i} '
        if zpart != '':
            circ_string += f'MZ({self.meas_error_rate}) ' + zpart + '\n'
        if xpart != '':
            circ_string = f'MX({self.meas_error_rate}) ' + xpart + '\n'
        if ypart != '':
            circ_string += f'MY({self.meas_error_rate}) ' + ypart + '\n'
        return stim.Circuit(circ_string)
        

    def stabilizer_gadget(self, stabilizer_in, ancilla_index, construction='cnot'):
        """
        Input:
            stabilizer: a single stabilizer written in terms of {I/_,X,Y,Z}
            construction: direct or hadamard:
                1) `cnot` using only CNOTs from data to ancilla along with single qubit gates
                    - H then S    : rotates Z basis -> Y basis
                    - S_dag then H: rotates Y basis -> Z basis
                    verifiable via checking that: Y stabilizer == kron(S@H,I) @ CNOT @ kron(H@S_dag)
                2) `hadamard` using H gates on ancilla and C-Pauli from ancilla to data
        Output:
            Measurement gadget

        the data qubits that are indicated in the stabilizer appear first
        the ancilla index starts from 0, which is the first ancilla qubit after the data 
        """

        # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = ''
        if construction == 'cnot':
            for i, pauli in enumerate(stabilizer):
                if pauli == 'Z':
                    # Z-gates are just cnots from data to ancilla
                    noise_string = self.noise_string_2qb() + f'{i} {N+ancilla_index}\n'
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string
                elif pauli == 'X':
                    # X-gates are conjugated by hadamards
                    noise_string_1qb = self.noise_string_1qb() + f'{i}\n'
                    noise_string_2qb = self.noise_string_2qb() + f'{i} {N+ancilla_index}\n'
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string_2qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                elif pauli == 'Y':
                    # Y-gates are conjugated by S-gates and hadamards
                    noise_string_1qb = self.noise_string_1qb() + f'{i}\n'
                    noise_string_2qb = self.noise_string_2qb() + f'{i} {N+ancilla_index}\n'
                    circ_string = f'S_DAG {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string_2qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'S {i} \n'
                    circ_string += noise_string_1qb 

            # noisy ancilla measurement
            circ_string += f'MR({self.meas_error_rate}) {N+ancilla_index}\n'
        return stim.Circuit(circ_string)
    
    def stabilizer_gadget_v2(self, stabilizer_in, ancilla_index, construction='cnot'):
        """
        Input:
            stabilizer: a single stabilizer written in terms of {I/_,X,Y,Z}
            construction: direct or hadamard:
                1) `cnot` using only CNOTs from data to ancilla along with single qubit gates
                    - H then S    : rotates Z basis -> Y basis
                    - S_dag then H: rotates Y basis -> Z basis
                    verifiable via checking that: Y stabilizer == kron(S@H,I) @ CNOT @ kron(H@S_dag)
                2) `hadamard` using H gates on ancilla and C-Pauli from ancilla to data
        Output:
            Measurement gadget

        the data qubits that are indicated in the stabilizer appear first
        the ancilla index starts from 0, which is the first ancilla qubit after the data 
        """

        # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = f'MPP({self.meas_error_rate}) '
        for i, pauli in enumerate(stabilizer):
            if (pauli != 'I'):
                circ_string += f'{pauli}{i}*'
        circ_string = circ_string[:-1] + '\n'
        return stim.Circuit(circ_string)
                
                
#                 bS, bH, mid, aH, aS = '','','','','' # before Sgate, before Hgate, mid gate, after Hgate, after Sgate
#                 for i, pauli in enumerate(stab):
#                     s = f'{i} '
#                     if pauli != 'I':
#                         mid += s + f'{ancilla+N} '
#                         if pauli == 'X':
#                             bH += s
#                             aH += s
#                         elif pauli == 'Y':
#                             bS += s
#                             bH += s
#                             aH += s
#                             aS += s
#                 # gate noise
#                 if gate_noise1:
#                     bS = f'S_DAG {bS} \n{gate_noise1} {bS}' if len(bS) > 0 else ''
#                     bH = f'H {bH}     \n{gate_noise1} {bH}' if len(bH) > 0 else ''
#                     aH = f'H {aH}     \n{gate_noise1} {aH}' if len(aH) > 0 else ''
#                     aS = f'S {aS}     \n{gate_noise1} {aS}' if len(aS) > 0 else ''
#                 else:
#                     bS = f'S_DAG {bS}' if len(bS) > 0 else ''
#                     bH = f'H {bH}    ' if len(bH) > 0 else ''
#                     aH = f'H {aH}    ' if len(aH) > 0 else ''
#                     aS = f'S {aS}    ' if len(aS) > 0 else ''

#                 if gate_noise2:
#                     mid = f'CX {mid} \n{gate_noise2} {mid}' if len(mid) > 0 else ''
#                 else:
#                     mid = f'CX {mid}' if len(mid) > 0 else ''

#                 # measurement noise
#                 if meas_noise:
#                     aS += f'\n{meas_noise} {ancilla + N}'
#                 aS += f'\nMR {ancilla + N}'
#                 gadgets.append(stim.Circuit('\n'.join([bS,bH,mid,aH,aS])))

#             elif construction == 'hadamard':
#                 before, mX, mY, mZ, after = '','','','',''
#                 s1 = f'{ancilla+N} '
#                 before += s1
#                 for i, pauli in enumerate(stab):
#                     s2 = f'{ancilla+N} {i} '
#                     if pauli == 'X': 
#                         mX += s2
#                     elif pauli == 'Y': 
#                         mY += s2
#                     elif pauli == 'Z': 
#                         mZ += s2
#                 after += s1 

#                 # gate noise
#                 if gate_noise1:
#                     before = f'H {before} \n{gate_noise1} {before}' if len(before) > 0 else ''
#                     after  = f'H {after}  \n{gate_noise1} {after}' if len(after) > 0 else ''
#                 else:
#                     before = f'H {before}' if len(before) > 0 else ''
#                     after  = f'H {after}' if len(after) > 0 else ''

#                 if gate_noise2:
#                     mX = f'CX {mX} \n{gate_noise2} {mX}' if len(mX) > 0 else ''
#                     mY = f'CY {mY} \n{gate_noise2} {mY}' if len(mY) > 0 else ''
#                     mZ = f'CZ {mZ} \n{gate_noise2} {mZ}' if len(mZ) > 0 else ''
#                 else:
#                     mX = f'CX {mX}' if len(mX) > 0 else ''
#                     mY = f'CY {mY}' if len(mY) > 0 else ''
#                     mZ = f'CZ {mZ}' if len(mZ) > 0 else ''

#                 # measurement noise
#                 if meas_noise:
#                     after += f'\n{meas_noise} {ancilla + N}'
#                 after += f'\nMR {ancilla + N}'
#                 gadgets.append(stim.Circuit('\n'.join([before,mX,mY,mZ,after])))

 #       return gadgets
        
        
        
    